In [169]:

USE ERP
GO
CREATE PROCEDURE usp_DropDatabase
    @DatabaseName NVARCHAR(128)
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @KillCommand NVARCHAR(MAX);
    DECLARE @Sql NVARCHAR(MAX);

    -- Initialize the command to kill active connections
    SET @KillCommand = '';

    -- Build the kill command for each active connection, excluding the current session
    SELECT @KillCommand = @KillCommand + 'KILL ' + CONVERT(NVARCHAR(10), session_id) + ';'
    FROM sys.dm_exec_sessions
    WHERE database_id = DB_ID(@DatabaseName) AND session_id <> @@SPID;

    -- Execute the kill commands
    EXEC sp_executesql @KillCommand;

    -- Drop the database
    SET @Sql = N'DROP DATABASE IF EXISTS [' + @DatabaseName + ']';
    EXEC sp_executesql @Sql;
END

GO

/*
Anlegen der grundsätzlichen Datenbankobjekte (Datenbank, Schema)
*/

-- Ausführen der Helper Stored Procedure um die Datenbank zu löschen
EXEC usp_DropDatabase 'dwh'

CREATE DATABASE dwh COLLATE SQL_Latin1_General_CP1_CI_AS; -- Gleiche Collation wie die ERP Datenbank


Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure usp_DropDatabase, Line 1
There is already an object named 'usp_DropDatabase' in the database.

Commands completed successfully.

Total execution time: 00:00:00.742

In [170]:
USE DWH;
GO

CREATE PROCEDURE Transfer_TopSeller
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50) = 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50) = 'Staging_TopSeller.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_TopSeller.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Sales.SalesTerritory',
    @DestinationTable3 VARCHAR(50) = 'Staging_TopSeller.SalesTerritory',
    @SourceTable4 VARCHAR(50) = 'Sales.SalesPerson',
    @DestinationTable4 VARCHAR(50) = 'Staging_TopSeller.SalesPerson',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;

        -- Vierte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable4;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable4;
        EXEC sp_executesql @SQL;


    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.165

In [171]:
USE DWH;
GO

CREATE PROCEDURE CreateSalespersonIDTable
AS
BEGIN
    SET NOCOUNT ON;

    BEGIN TRY
        -- Überprüfen, ob die Tabelle existiert, und Löschen, falls sie vorhanden ist
        IF OBJECT_ID('DWH.DWH_TopSeller.Dim_SalesPersonID', 'U') IS NOT NULL
        BEGIN
            DROP TABLE DWH.DWH_TopSeller.Dim_SalesPersonID; 
        END

        -- Erstellen der Tabelle Dim_SalesPersonID mit den Spalten SalesPersonID und SalesPersonName
        CREATE TABLE [DWH].[DWH_TopSeller].Dim_SalesPersonID (
            SalesPersonID INT PRIMARY KEY NOT NULL,      
            SalesPersonName VARCHAR(100)
        );

        -- Einfügen der Werte in die Tabelle
        INSERT INTO [DWH].[DWH_TopSeller].Dim_SalesPersonID (SalesPersonID, SalesPersonName)
        VALUES 
            (NULL, NULL)
            (279, 'Stephen Oswald'),
            (277, 'John Doe'),
            (283, 'Alice Smith'),
            (276, 'Bob Johnson'),
            (282, 'Charlie Brown'),
            (275, 'Diana White'),
            (290, 'Eve Black'),
            (281, 'Frank Green'),
            (288, 'Grace Lee'),
            (289, 'Harry Martin'),
            (280, 'Irene Davis'),
            (286, 'Jack Wilson');
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.170

In [172]:
USE DWH;
GO
CREATE PROCEDURE Transfer_SalesGrowth
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50) = 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Production.Product',
    @DestinationTable3 VARCHAR(50) = 'Staging_SalesGrowth.Product',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.144

In [173]:
USE DWH;
GO

CREATE PROCEDURE ETL_SalesGrowth_ProductNumber
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_SalesGrowth.Dim_ProductNumber';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY
        -- Drop the table if it exists
        IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @TableToCreate;
            EXEC sp_executesql @SQL;
        END

        -- Create the new table with ProductID and ProductNumber as a primary key
        SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
            ProductID INT,
            ProductNumber VARCHAR(255) NOT NULL, 
            PRIMARY KEY (ProductNumber)
        )';
        EXEC sp_executesql @SQL;

        -- Insert distinct ProductID and ProductNumber values
        SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (ProductID, ProductNumber)
        SELECT DISTINCT pr.ProductID, 
        pr.ProductNumber
        FROM  
            Staging_SalesGrowth.SalesOrderHeader sh  
        LEFT OUTER JOIN  
            Staging_SalesGrowth.SalesOrderDetail sd ON sh.SalesOrderID = sd.SalesOrderID 
        LEFT OUTER JOIN  
            Staging_SalesGrowth.Product pr ON sd.ProductID = pr.ProductID';
        EXEC sp_executesql @SQL;

    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.095

In [174]:
USE DWH;
GO

CREATE PROCEDURE ETL_SalesGrowth_ProductID
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_SalesGrowth.Dim_ProductID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        ProductID INT PRIMARY KEY,
        Produktname VARCHAR(MAX) 
    )';
    EXEC sp_executesql @SQL;

-- Define the SELECT DISTINCT AND THE COLUMNS we want to select
    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (ProductID, Produktname)
    SELECT DISTINCT pr.ProductID, 
pr.Name as Produktname
FROM  
    Staging_SalesGrowth.SalesOrderHeader sh  
LEFT OUTER JOIN  
    Staging_SalesGrowth.SalesOrderDetail sd ON sh.SalesOrderID = sd.SalesOrderID 
LEFT OUTER JOIN  
    Staging_SalesGrowth.Product pr ON sd.ProductID = pr.ProductID';
    EXEC sp_executesql @SQL;

    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.103

In [175]:
USE DWH;
GO

CREATE PROCEDURE "ETL_TopSeller_TerritoryID"
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_TopSeller.Dim_TerritoryID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        territoryID INT PRIMARY KEY,
        Territory VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (territoryID, Territory)
    SELECT DISTINCT sap.territoryID, 
st.Name as Territory
from Staging_TopSeller.SalesPerson sap 
left outer join Staging_TopSeller.SalesTerritory st on st.territoryID = sap.TerritoryID
WHERE sap.territoryID IS NOT NULL';
    EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.102

In [176]:
USE DWH;
GO

CREATE PROCEDURE ETL_SalesGrowth_Facttable
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_SalesGrowth.Facttable';
    DECLARE @SQL NVARCHAR(MAX);
    
    BEGIN TRY
        -- Falls die Tabelle bereits existiert, lösche sie
        IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
        BEGIN
            DECLARE @DropFKSQL NVARCHAR(MAX);
            SET @DropFKSQL = N'';

            SELECT @DropFKSQL = @DropFKSQL + 'ALTER TABLE ' + QUOTENAME(OBJECT_NAME(parent_object_id)) + 
                                ' DROP CONSTRAINT ' + QUOTENAME(name) + '; '
            FROM sys.foreign_keys
            WHERE referenced_object_id = OBJECT_ID(@TableToCreate);

            EXEC sp_executesql @DropFKSQL;

            SET @SQL = 'DROP TABLE ' + @TableToCreate;
            EXEC sp_executesql @SQL;
        END

        -- Erstelle die neue Tabelle mit ProductID und ProductNumber
        SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
            ProductID INT NOT NULL,
            ProductNumber VARCHAR(255) NOT NULL,
            Umsatz_Vorjahr DECIMAL(18, 6) NULL,
            Umsatz_Betrachtungsjahr DECIMAL(18, 6) NULL,
            Wachstum DECIMAL(18, 6) NULL,
            CONSTRAINT FK_ProductID FOREIGN KEY (ProductID) REFERENCES DWH_SalesGrowth.Dim_ProductID(ProductID),
            CONSTRAINT FK_ProductNumber FOREIGN KEY (ProductNumber) REFERENCES DWH_SalesGrowth.Dim_ProductNumber(ProductNumber)
        )';
        EXEC sp_executesql @SQL;

        -- Erstelle die Insert-Abfrage
        SET @SQL = '
        ;WITH AktuellerUmsatz AS (
            SELECT 
                p.ProductID,
                p.ProductNumber,
                SUM(sod.OrderQty * sod.UnitPrice) AS UmsatzAktuell
            FROM Staging_SalesGrowth.SalesOrderDetail sod
            JOIN Staging_SalesGrowth.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
            JOIN Staging_SalesGrowth.Product p ON sod.ProductID = p.ProductID
            WHERE YEAR(soh.OrderDate) = 2014
            GROUP BY p.ProductID, p.ProductNumber
        ),
        VorjahresUmsatz AS (
            SELECT 
                p.ProductID,
                SUM(sod.OrderQty * sod.UnitPrice) AS UmsatzVorjahr
            FROM Staging_SalesGrowth.SalesOrderDetail sod
            JOIN Staging_SalesGrowth.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
            JOIN Staging_SalesGrowth.Product p ON sod.ProductID = p.ProductID
            WHERE YEAR(soh.OrderDate) = 2013
            GROUP BY p.ProductID
        )
        INSERT INTO ' + @TableToCreate + ' (ProductID, ProductNumber, Umsatz_Vorjahr, Umsatz_Betrachtungsjahr, Wachstum)
        SELECT 
            a.ProductID,
            a.ProductNumber,
            v.UmsatzVorjahr,
            a.UmsatzAktuell AS Umsatz_Betrachtungsjahr,
            (a.UmsatzAktuell - COALESCE(v.UmsatzVorjahr, 0)) AS Wachstum
        FROM AktuellerUmsatz a
        LEFT JOIN VorjahresUmsatz v ON a.ProductID = v.ProductID;
        ';

        EXEC sp_executesql @SQL;
    
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.103

In [177]:
USE DWH;
GO

CREATE PROCEDURE ETL_TopSeller_Facttable
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_TopSeller.Facttable';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY
        -- Falls die Tabelle bereits existiert, lösche sie
        IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
        BEGIN
            DECLARE @DropFKSQL NVARCHAR(MAX);
            SET @DropFKSQL = N'';

            SELECT @DropFKSQL = @DropFKSQL + 'ALTER TABLE ' + QUOTENAME(OBJECT_NAME(parent_object_id)) + 
                                ' DROP CONSTRAINT ' + QUOTENAME(name) + '; '
            FROM sys.foreign_keys
            WHERE referenced_object_id = OBJECT_ID(@TableToCreate);

            EXEC sp_executesql @DropFKSQL;

            -- Lösche die Tabelle
            SET @SQL = 'DROP TABLE ' + @TableToCreate;
            EXEC sp_executesql @SQL;
        END

        -- Erstelle die neue Tabelle
        SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
            SalesPersonID INT FOREIGN KEY REFERENCES DWH_TOPSELLER.DIM_SalesPersonID(SalesPersonID),
            TerritoryID INT FOREIGN KEY REFERENCES DWH_TOPSELLER.DIM_TerritoryID(TerritoryID),
            TotalSales INT,
        )';
        EXEC sp_executesql @SQL;

        -- Berechne GesamtUmsatz und füge Daten ein
        SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (SalesPersonID, TerritoryID, TotalSales)
        SELECT 
            soh.SalesPersonID,
            soh.TerritoryID,
            SUM(sod.OrderQty * sod.UnitPrice) AS TotalSales
        FROM Staging_TopSeller.SalesOrderDetail sod
        JOIN Staging_TopSeller.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
        WHERE soh.SalesPersonID IS NOT NULL
        GROUP BY soh.SalesPersonID, soh.TerritoryID;
        ';
        
        EXEC sp_executesql @SQL;
    
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.106

Commands completed successfully.

Total execution time: 00:00:00

In [179]:
CREATE SCHEMA Staging_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.072

In [180]:
CREATE SCHEMA Staging_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.074

In [181]:
CREATE SCHEMA DWH_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.072

In [182]:
CREATE SCHEMA DWH_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.052

In [183]:
EXEC Transfer_TopSeller;
EXEC Transfer_SalesGrowth
EXEC CreateSalespersonIDTable
EXEC ETL_SalesGrowth_ProductID
EXEC ETL_TopSeller_TerritoryID
EXEC ETL_SalesGrowth_ProductNumber
EXEC ETL_SalesGrowth_Facttable

(31465 rows affected)

(121317 rows affected)

(10 rows affected)

(17 rows affected)

(31465 rows affected)

(121317 rows affected)

(504 rows affected)

Total execution time: 00:00:02.138

In [184]:
EXEC ETL_TopSeller_Facttable

Ein Fehler ist aufgetreten: The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Facttable__Sales__4F7CD00D". The conflict occurred in database "dwh", table "DWH_TopSeller.Dim_SalesPersonID", column 'SalesPersonID'.

: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Facttable__Sales__4F7CD00D". The conflict occurred in database "dwh", table "DWH_TopSeller.Dim_SalesPersonID", column 'SalesPersonID'.

Total execution time: 00:00:00.312